# FLUX LoRA Training in Google Colab

이 노트북은 Google Colab에서 FLUX LoRA 훈련을 위한 환경설정 및 실행 가이드입니다.

In [ ]:
# GPU 정보 확인
!nvidia-smi
!nvcc --version

In [ ]:
# 필요한 패키지 설치
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install diffusers transformers peft accelerate datasets
!pip install Pillow pandas numpy tqdm wandb safetensors
!pip install sentencepiece protobuf python-dotenv

In [ ]:
# 환경변수 설정
import os
from google.colab import userdata

# 방법 1: Colab Secrets 사용 (권장)
# 🔑 키 아이콘 클릭 → Add new secret:
# Name: HUGGINGFACE_TOKEN
# Value: hf_your_token_here
try:
    HF_TOKEN = userdata.get('HUGGINGFACE_TOKEN')
    os.environ['HUGGINGFACE_TOKEN'] = HF_TOKEN
    print("✅ Token loaded from Colab secrets")
except:
    # 방법 2: 직접 입력 (임시용)
    HF_TOKEN = "hf_your_token_here"  # 여기에 토큰 입력
    os.environ['HUGGINGFACE_TOKEN'] = HF_TOKEN
    print("⚠️ Token set manually")

# 기타 환경변수 설정
os.environ['FLUX_MODEL_NAME'] = 'black-forest-labs/FLUX.1-schnell'
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'
print("✅ Environment variables configured")

In [ ]:
# 훈련 코드 다운로드
!git clone https://github.com/comking2/flux-lora-training.git
%cd flux-lora-training

# 또는 GitHub에서 개별 파일 다운로드
# !wget https://raw.githubusercontent.com/comking2/flux-lora-training/main/train_lora.py
# !wget https://raw.githubusercontent.com/comking2/flux-lora-training/main/dataset_processor.py
# !wget https://raw.githubusercontent.com/comking2/flux-lora-training/main/inference.py

In [ ]:
# 데이터셋 업로드 또는 다운로드
# 방법 1: Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 방법 2: 직접 업로드
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# GPU 메모리 최적화 설정
import torch

# GPU 메모리 확인
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")
    
    # 메모리 캐시 정리
    torch.cuda.empty_cache()

In [ ]:
# FLUX LoRA 훈련 실행
# T4 (16GB) 환경 최적화

!python train_lora.py \
    --data_dir "/content/drive/MyDrive/training_data" \
    --output_dir "./flux_lora_output" \
    --epochs 10 \
    --batch_size 1 \
    --lora_rank 8 \
    --learning_rate 5e-5